In [1]:
import gym
from collections import deque, namedtuple
import random
import numpy as np
from tqdm.notebook import tqdm 
import time

import pandas as pd
import torch
import torch
from torch import nn
from torch.optim import SGD, Adam
from torch.distributions.categorical import Categorical

from dataclasses import dataclass, field

import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme()
matplotlib.rcParams['figure.figsize'] = (20, 10)

from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Any, Optional, Callable

import toolz

import pickle

import random

In [2]:
class Identity(nn.Module):
    def forward(self, x): return x

class Argmax(nn.Module):
    def forward(self, x): return x.argmax()

def create_model(input_size: int, output_size: int, hidden_layers=[16], activate_every=2, output_layer=Identity):
    layers = [nn.Linear(in_shape, out_shape) for (in_shape, out_shape) in toolz.sliding_window(2, hidden_layers)]
    layers = [*toolz.concat([*ls, nn.ReLU()] for ls in toolz.partition(activate_every, layers))]
    layers = [nn.ReLU(), *layers]
    return nn.Sequential(
        nn.Linear(input_size, hidden_layers[0]),
        *layers,
        nn.Linear(hidden_layers[-1], output_size),
        output_layer(),
    )

In [3]:
@dataclass
class Model:
    input_size     : int
    output_size    : int
    hidden_layers  : List[int]
    activate_every : int
    model          : nn.Module                  
    create_model_fn: Callable[[Any], nn.Module] 
    output_layer   : nn.Module                  = field(default_factory=Identity)
    
    @classmethod
    def from_input_size(
        cls, 
        input_size: int, 
        output_size: int, 
        hidden_layers: List[int]=[8], 
        activate_every: int=2, 
        output_layer: 
        nn.Module=Identity
    ):
        def create_model_fn():
            model = create_model(
                input_size=input_size, 
                output_size=output_size, 
                hidden_layers=hidden_layers,
                activate_every=activate_every,
                output_layer=output_layer,
            )
            model.train()
            for p in model.parameters(): p.requires_grad_(False)
            return model
        
        return cls(
            input_size=input_size, 
            output_size=output_size, 
            hidden_layers=hidden_layers,
            activate_every=activate_every,
            output_layer=output_layer,
            create_model_fn=create_model_fn,
            model=create_model_fn(),
        )
    
        
    def __call__(self, x): return self.model(x)
    def get_action(self, x): return self(x).item()
    
    def jiggle(self, mu=0, sigma=1.0):
        model  = pickle.loads(pickle.dumps(self.model))
        for p in model.parameters():
            p.set_(p + (mu + sigma * torch.randn(*p.shape)))
        
        return Model(
            input_size      = self.input_size, 
            output_size     = self.output_size, 
            hidden_layers   = self.hidden_layers,
            activate_every  = self.activate_every,
            output_layer    = self.output_layer,
            create_model_fn = self.create_model_fn,
            model           = model,
        )
    
    def step(self, env: gym.Env, fps: Optional[int]=None, show_progress: bool=True) -> Dict[str, Any]:
        state = env.reset()
        done = False
        ep_len = 0
        total_reward = 0
        
        while not done:
            if fps: env.render()
            action = self.get_action(torch.as_tensor(state, dtype=torch.float32))
            state, reward, done, _ = env.step(action)
            if fps: time.sleep(1/fps)
            
            total_reward += reward
            ep_len += 1
            
        if fps: env.close()
        return { 
            'EpisodeLength': ep_len, 
            'Reward': total_reward,
        }

ModelScore = Tuple[Model, float]
class SelectionPolicy(ABC):
    @abstractmethod
    def select(xs: List[ModelScore]): raise NotImplementedError()
    
class CrossoverPolicy(ABC):
    @abstractmethod
    def crossover(xs: List[Model], lr: float, target_size: Optional[int]): raise NotImplementedError()

class LRPolicy(ABC):
    @abstractmethod
    def update(self, lr: float): raise NotImplementedError()

class StopPolicy(ABC):
    def update(self, rewards: List[float]) -> bool: raise NotImplementedError()

    
@dataclass
class ConstStop(StopPolicy):
    value: int = 150
    
    def update(self, rewards: List[float]) -> bool: 
        return sum(rewards) / len(rewards) > self.value
    
@dataclass
class ConstLR(LRPolicy):
    lr: float=0.1
    def update(self): return self.lr

@dataclass
class DecayingLR(LRPolicy):
    decay_factor: float = 0.99
    lr          : float = 0.1
    def update(self): 
        self.lr = self.lr * self.decay_factor
        return self.lr

@dataclass
class TopPctPolicy(SelectionPolicy):
    pct: float = 0.5
    def select(self, xs: List[ModelScore]) -> List[ModelScore]:
        scores = sorted(xs, key=lambda s: s[1], reverse=True)
        return scores[:int(len(scores) // (1 / self.pct))]

@dataclass
class TournamentPolicy(SelectionPolicy):
    tournament_size: int = 2
    def select(self, xs: List[ModelScore]) -> List[ModelScore]:
        random.shuffle(xs) 
        
        winners = []
        for individuals in toolz.partition_all(self.tournament_size, xs):
            winner = max(individuals, key=lambda score: score[1])
            winners.append(winner)
        
        return winners
    
@dataclass
class Jiggle(CrossoverPolicy):
    keep: bool
    def crossover(self, models: List[Model], lr: float, target_size: Optional[int]=None) -> List[Model]: 
        target_size = len(models) if target_size is None else target_size
        
        ratio = target_size // len(models)
        
        if self.keep:
            ms = []
            for model in models:
                descendants = [model.jiggle(mu=0.0, sigma=lr) for i in range(ratio)]
                ms.append(descendants)
        else:
            for model in models:
                descendants = [model.jiggle(mu=0.0, sigma=lr) for i in range(ratio) - 1]
                descendants.append(model)
                ms.append(descendants)
        return [m for mm in ms for m in mm] # Flatten list
        

In [4]:
@dataclass
class Solver:
    population      : List[Model]
    env             : gym.Env
    lr_policy       : LRPolicy        = field(default_factory=ConstLR)
    selection_policy: SelectionPolicy = field(default_factory=TopPctPolicy)
    crossover_policy: CrossoverPolicy = field(default_factory=lambda: Jiggle(keep=True))
    stop_policy     : StopPolicy      = field(default_factory=ConstStop)
    env_name        : str             = 'CartPole-v1'
    population_size : int             = 50
    
    @classmethod
    def from_params(
        cls, 
        env_name: str        = 'CartPole-v1', 
        population_size: int = 50,
        lr: float            = 0.05,
        **kwargs
    ):
        env = gym.make(env_name)
        input_size = env.observation_space.shape[0]
        
        
        action_space = env.action_space
        if isinstance(action_space, gym.spaces.Box):
            output_size = action_space.shape[0]
            output_layer = Identity
        elif isinstance(action_space, gym.spaces.Discrete):
            output_size = action_space.n
            output_layer = Argmax       
        else: raise ValueError(f'Unkown action space for env {env_name}')
        
        population = [Model.from_input_size(
            input_size=input_size, 
            output_size=output_size,
            output_layer=output_layer,
        ) for _ in range(population_size)]
        
        return cls(
            population=population,
            env=env,
            env_name=env_name,
            population_size=population_size,
            **kwargs
        )
        
    def step(self, step: int) -> Dict[str, Any]:
        results = []
        for i, model in enumerate(self.population):
            result = model.step(env=self.env)
            result['step'] = step
            result['model_number'] = i
            result['model'] = model
            results.append({ **result })

        population = self.selection_policy.select([(row['model'], row['Reward']) for row in results])
        population = [model for model, _reward in population]
        population = self.crossover_policy.crossover(population, self.lr_policy.update(), target_size=self.population_size)
        self.population = population
        
        return [toolz.keyfilter(lambda k: k != 'model', r) for r in results]
    
    def train(self, steps: int, show_progress: bool=True) -> pd.DataFrame:
        results = []
        
        loader = range(steps)
        loader = tqdm(loader) if show_progress else loader
        for step in loader:
            result = self.step(step)

            results.extend(result)
            loader.set_postfix({ 
                'lr': self.lr_policy.lr, 
                'reward': sum([r['Reward'] for r in result])/ len(result),
                'pop_size': len(self.population)})

            stop = self.stop_policy.update([r['Reward'] for r in result])
            if stop: break
            
        return pd.DataFrame(results)

In [5]:
solver = Solver.from_params(
    lr_policy = DecayingLR(decay_factor=0.99, lr=0.5),
    #selection_policy = TournamentPolicy(tournament_size=2),
    selection_policy = TopPctPolicy(pct=0.1),
)

In [6]:
df = solver.train(50).sort_values('Reward', ascending=False)
df.loc[df['step'] == df['step'].max()].head(5)

  0%|          | 0/50 [00:00<?, ?it/s]

TypeError: object of type 'int' has no len()

In [11]:
_ = sns.lineplot(data=df, x='step', y='Reward')

In [31]:
best_model_idx = df.loc[df['step'] == df['step'].max()]['Reward'].idxmax()
best_model_row = df.loc[best_model_idx]
best_model = solver.population[int(best_model_row['model_number'])]

In [33]:
best_model.step(env=solver.env, fps=30)

{'EpisodeLength': 77, 'Reward': 77.0}